# SSSP

In [ ]:
# Instalamos PySpark
!pip install pyspark

In [1]:
from pyspark.sql import SparkSession

In [2]:

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

rdd_nodes = spark.sparkContext.parallelize(nodes)
rdd_edges = spark.sparkContext.parallelize(edges)


In [ ]:
# Creamos nodos y aristas aleatorias para usarlo en el algoritmo
import random

numero_nodos = 20

numero_aristas = 50

nodes = [int(i) for i in range(numero_nodos)]

edges = []
for _ in range(numero_aristas):
    nodo_inicial = random.randint(0, numero_nodos)
    nodo_destino = random.randint(0, numero_nodos)
    peso = random.randint(1, 40)
    arista = (nodo_inicial, nodo_destino, peso)
    edges.append(arista)

In [ ]:
print(nodes)
print(edges)

In [ ]:
# Paralelizamos
rdd_nodes = spark.sparkContext.parallelize(nodes)
rdd_edges = spark.sparkContext.parallelize(edges)

# SELECCIÓN DEL NODO INICIAL

In [ ]:
# Seleccionamos un nodo incial (Tener ojo con la seleccion, si por algun motivo este nodo no esta conectado a ningun otro, va a dar infinito en todos los caminos, debido
# a que no existe un camino entre ese nodo y otro)
nodo_inicial = 3

---

In [ ]:
# Filtramos los caminos que salen de ese nodo
rdd_caminos_iniciales = rdd_edges.filter(lambda x: x[0] == nodo_inicial)

In [ ]:
# inicialicemos los costes de los nodos a infinito y el coste del nodo origen a 0
rdd_costs = rdd_nodes.map(lambda x: (x, float('inf')))
rdd_costs = rdd_costs.map(lambda x: (x[0], 0.0) if x[0] == nodo_inicial else x)

In [ ]:
# realicemos lo anterior de manera iterativa hasta que no existan caminos que mejoren el coste de llegada a los nodos destino
# creemos un ciclo que se repita hasta que no existan caminos que mejoren el coste de llegada a los nodos destino
while True:
    # veamos los caminos que encontramos desde el nodo_destino de los caminos iniciales

    rdd_costs_ac = rdd_edges.map(lambda x: (x[0],(x))).join(rdd_caminos_iniciales.map(lambda x: (x[1], (x[0],x[1],x[2]))))
    rdd_caminos = rdd_caminos_iniciales.union(rdd_costs_ac.map(lambda x: (x[1][1][0],x[1][0][1],x[1][1][2] + x[1][0][2])))

    # actualicemos los costes de los nodos destino si el costo del camino encontrado es menor que el costo del nodo destino
    # unamos los rdd y luego hagamos un reduceByKey para quedarnos con el menor coste

    rdd_costs_nuevos = rdd_costs.union(rdd_caminos.map(lambda x: (x[1], x[2])))
    rdd_costs_nuevos = rdd_costs_nuevos.reduceByKey(lambda x,y: x if x < y else y)

    # veamos si los costes encontrados son iguales a los ya encontrados
    if rdd_costs_nuevos.join(rdd_costs).filter(lambda x: x[1][0] != x[1][1]).count() == 0:
        break

    # actualicemos los costes iniciales con los costes encontrados
    rdd_costs = rdd_costs_nuevos
    # actualicemos los caminos iniciales con los caminos encontrados
    rdd_caminos_iniciales = rdd_caminos

In [ ]:
rdd_costs.collect()

### Una consideración

Lo llegamos a escalar con 70 nodos y como 150 aristas, corría el programa y todo, se demoraba su rato piola, pero de que funciona, funciona 